# Seminar Python 3: Pandas - data processing and statistics

Converting dates from string

In [ ]:
#Example 1. Convert the DATE column from string to date
import dateutil
import pandas as pd
df = pd.read_csv('phone_data.csv')
print(df.dtypes)
df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=True)
print(df.dtypes)

Grouping and applying functions to groups

In [ ]:
#Example 2. Grouping records
import pandas as pd
df = pd.read_csv('phone_data.csv')
print(df.groupby(['item']).groups.keys())
print(len(df.groupby(['item']).groups['call']))

import pandas as pd
df = pd.read_csv('phone_data.csv')
print(df.groupby(['month']).groups.keys())
print(len(df.groupby(['month']).groups['2014-11']))

In [ ]:
#Example 3. Applying functions by groups
import pandas as pd
df = pd.read_csv('phone_data.csv')
print('First record from each group, by item') #???
print(df.groupby('item').first())

print('Total of duration for each month')
print(df.groupby('month')['duration'].sum())

print('Total duration for calls, by network')
print(df[df['item'] == 'call'].groupby('network')['duration'].sum())

In [ ]:
#Example 4. Complex grouping
import pandas as pd
df = pd.read_csv('phone_data.csv')
print('Count of calls, sms, data transfer for each month')
print(df.groupby(['month', 'item'])['date'].count())

In [ ]:
#Example 5. Grouping and data aggregation
import pandas as pd
df = pd.read_csv('phone_data.csv')
# Group by month and item; compute statistics for each group
print(df.groupby(['month', 'item']).agg({'duration':sum,      # sum of durations per group
                                     'network_type': "count", # count of network types
                                     'date': 'first'}))    # first value (data) for each group

df1 = df.groupby(['month', 'item']).agg({'duration':sum,      # sum of durations per group
                                     'network_type': "count", # count of network types
                                     'date': 'first'})
df1.to_csv('aggregate.csv')

In [ ]:
#Example 6. Applying multiple functions per column
import pandas as pd
df = pd.read_csv('phone_data.csv')
# Group by month and item; compute statistics for each group
print(df.groupby(['month', 'item']).agg({'duration': [min, max, sum],      # min, max, sum of duration
                                     'network_type': "count", # count of network types
                                     'date': [min, 'first', 'nunique']}))  # min, first, no. of unique values per group

DataFrame merge and join operations

The csv data files **user_device.csv** and **user_usage.csv** have the common column **use_id**.  
The column **Model** in **supported_devices.csv** has common values with the **device** column in **user_device.csv**.

In [ ]:
#Example 7. Inner merge / inner join
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id')
print(result)
print('Dataset structure user_usage.csv ', df.shape)
print('Dataset structure user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Example 8. Left merge / left join
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='left')
print(result)
print('Dataset structure user_usage.csv ', df.shape)
print('Dataset structure user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Example 9. Right merge / right join
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='right')
print(result)
print('Dataset structure user_usage.csv ', df.shape)
print('Dataset structure user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Example 10. Full outer merge / full outer join
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='outer')
print(result)
print('Dataset structure user_usage.csv ', df.shape)
print('Dataset structure user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Example 11. Full outer merge / full outer join with the _merge indication
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='outer',
                  indicator=True)
print(result)
print('Dataset structure user_usage.csv ', df.shape)
print('Dataset structure user_device.csv ', df1.shape)
print(df['use_id'].isin(df1['use_id']).value_counts())

In [ ]:
#Example 12. Merge with three datasets

import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')
df3 = pd.read_csv('supported_devices.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='left')

df3.rename(columns={"Retail Branding": "manufacturer"}, inplace=True)
result = pd.merge(result,
                  df3[['manufacturer', 'Model']],
                  left_on='device',
                  right_on='Model',
                  how='left')
print(result.head())
print(result.shape)

In [ ]:
#Example 13. Applying groupby and agg on the resulting dataset
import pandas as pd
df = pd.read_csv('user_usage.csv')
df1 = pd.read_csv('user_device.csv')
df3 = pd.read_csv('supported_devices.csv')

result = pd.merge(df,
                  df1[['use_id', 'platform', 'device']],
                  on='use_id',
                  how='left')

df3.rename(columns={"Retail Branding": "manufacturer"}, inplace=True)
result = pd.merge(result,
                  df3[['manufacturer', 'Model']],
                  left_on='device',
                  right_on='Model',
                  how='left')
print(result.groupby("manufacturer").agg({
    "outgoing_mins_per_month": "mean",
    "outgoing_sms_per_month": "mean",
    "monthly_mb": "mean",
    "use_id": "count"
}))

# Using *matplotlib* to create charts

In [ ]:
#Example 14. Create a bar chart with matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.max_columns",10)
df = pd.read_csv('clients_leasing20.csv')
print(df['AGE'])
df['AGE'].plot(kind='bar')
plt.xlabel('ID_CLIENT')
plt.ylabel('AGE')
plt.show()

In [ ]:
#Example 15. Applying colors to bars, according to bar percentage
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
pd.set_option("display.max_columns",10)
df = pd.read_csv('clients_leasing20.csv')
print(df['AGE'])
#all_colors = list(plt.cm.colors.cnames.keys())
colors = cm.hsv( df['AGE']/ float(max(df['AGE'])))
df['AGE'].plot(kind='bar',color=colors)
plt.xlabel('ID_CLIENT')
plt.ylabel('AGE')
plt.show()

In [ ]:
#Example 16. Create a histogram with matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.max_columns",10)
df = pd.read_csv('clients_leasing20.csv')
print(df['AGE'])
df['AGE'].plot(kind='hist')
plt.xlabel('AGE')
plt.show()

In [ ]:
#Example 17. Create a bar chart for groups of records, sorting
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('clients_leasing20.csv')
plot_data=df[df['SEX']=='m']
plot_data=plot_data.groupby('JOB')['INCOME_PER_YEAR'].sum()
plot_data.sort_values().plot(kind='bar')
plt.show()

# References
https://matplotlib.org/index.html  
https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/  
https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

Exercises
1) Create a pie chart for showing the SMS (duration ?) by month, using **phone_data.csv**.

2) Create a dataset from **user_usage.csv** and **supported_devices.csv**;  
create a bar chart with the total data traffic (**monthly_mb**) for each brand in column **Retail Branding**.

3) Use **phone_data.csv** to compute the total duration for each month, and the total duration per network type (col. **mobile**) for each month.